In [2]:
import os, random
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix

random.seed(10)


In [3]:
os.getcwd()
data_path = "/defects/src/data/Jureczko/collated_data/"
os.chdir(os.getcwd() + data_path)

In [5]:
projList = os.listdir()
print("Files:",projList)
projs = [p.split('_')[0] for p in projList]
print("List of projects:",projs)

Files: ['ant_merged.csv', 'camel_merged.csv', 'ivy_merged.csv', 'jedit_merged.csv', 'log4j_merged.csv', 'lucene_merged.csv', 'poi_merged.csv', 'velocity_merged.csv', 'xalan_merged.csv', 'xerces_merged.csv']
List of projects: ['ant', 'camel', 'ivy', 'jedit', 'log4j', 'lucene', 'poi', 'velocity', 'xalan', 'xerces']


In [38]:
df_n = pd.read_csv("..//samplingResults_n.csv")
fractions = []
for i in range(10):
    for j in range(10):
        fractions.append(df_n.iloc[i][j+1]/100)
len(fractions)

100

In [39]:
tbl = [[0]*len(projs) for p in projs]
count=-1
for i in range(len(projs)):
    print("\nIteration ",i)
    trainData = pd.read_csv(projList[i])
    
    for j in range(len(projs)):
        count+=1
        if (i != j):
            #print("Here: ",float(df_n.iloc[i][j+1]/100))
            trainData.sample(frac=fractions[count])
            X_train = trainData.loc[:,trainData.columns!='$<bug']
            y_train = trainData.loc[:,trainData.columns=='$<bug']
            
            testData = pd.read_csv(projList[j])
            X_test = testData.loc[:, testData.columns!='$<bug']
            y_test = testData.loc[:, testData.columns=='$<bug']
            clf = RandomForestClassifier(n_estimators=1000, n_jobs=1)
            clf.fit(X_train, y_train.values.ravel())
            y_pred = clf.predict(X_test)
            cm = confusion_matrix(y_test, y_pred)
            recall = cm[1][1]/(cm[1][1] + cm[1][0])
            pf = cm[0][1]/(cm[0][1] + cm[0][0])
            
            g = 2/((1/recall) + (1/(1-pf)))
            
            print("Model trained on",projs[i],", tesing on",projs[j])
            #acc = metrics.accuracy_score(y_test, y_pred)
            #print("Project: {}, Accuracy: {}, Precision: {}".format(projs[j],acc,
            #                                                       metrics.precision_score(y_test, y_pred)))
            tbl[i][j]=g
            #print("Test Project: {}, G-Score {}".format(projs[j], g))
print(tbl)


Iteration  0
Model trained on ant , tesing on camel
Model trained on ant , tesing on ivy
Model trained on ant , tesing on jedit
Model trained on ant , tesing on log4j
Model trained on ant , tesing on lucene
Model trained on ant , tesing on poi
Model trained on ant , tesing on velocity
Model trained on ant , tesing on xalan
Model trained on ant , tesing on xerces

Iteration  1
Model trained on camel , tesing on ant
Model trained on camel , tesing on ivy
Model trained on camel , tesing on jedit
Model trained on camel , tesing on log4j
Model trained on camel , tesing on lucene
Model trained on camel , tesing on poi
Model trained on camel , tesing on velocity
Model trained on camel , tesing on xalan
Model trained on camel , tesing on xerces

Iteration  2
Model trained on ivy , tesing on ant
Model trained on ivy , tesing on camel
Model trained on ivy , tesing on jedit
Model trained on ivy , tesing on log4j
Model trained on ivy , tesing on lucene
Model trained on ivy , tesing on poi
Model t

In [40]:
df = pd.DataFrame(tbl)
df.columns=projs
df.insert(0,'projects',projs)
print(df)
#print(os.getcwd())
df.to_csv("..//post_sampling_gscore.csv",index=False)

   projects       ant     camel       ivy     jedit     log4j    lucene  \
0       ant  0.000000  0.101226  0.385596  0.582279  0.109059  0.182377   
1     camel  0.398118  0.000000  0.375421  0.505455  0.242442  0.228678   
2       ivy  0.209302  0.034955  0.000000  0.276655  0.037732  0.123990   
3     jedit  0.157744  0.038393  0.111026  0.000000  0.007663  0.009091   
4     log4j  0.257380  0.338042  0.227812  0.236897  0.000000  0.355090   
5    lucene  0.462258  0.516806  0.533795  0.483632  0.637914  0.000000   
6       poi  0.645508  0.570165  0.649033  0.617694  0.621242  0.624541   
7  velocity  0.550314  0.502988  0.458872  0.447936  0.512429  0.454270   
8     xalan  0.602899  0.526973  0.587643  0.466730  0.583352  0.578249   
9    xerces  0.414248  0.349195  0.429603  0.392750  0.526491  0.525201   

        poi  velocity     xalan    xerces  
0  0.139191  0.093391  0.254520  0.217013  
1  0.202408  0.177863  0.259373  0.246078  
2  0.070911  0.047851  0.147351  0.162521 